In [1]:
'''第一问的代码'''
import networkx as nx
import pandas as pd
origin_data = pd.read_csv("influence_data.csv",encoding="utf-8")
Share={
    'Pop/Rock':0.8080700725646852,
    'R&B;':0.7882599580712788,
    'Country':0.770320197044335,
    'Jazz':0.8320790216368767,
    'Electronic':0.33214285714285713,
    'Vocal':0.6423444976076556,
    'Reggae':0.7849462365591398,
    'Latin':0.569620253164557,
    'Folk':0.43851132686084143,
    'Blues':0.7005649717514124,
    'Religious':0.45722713864306785,
    'International':0.31153846153846154,
    'New Age':0.24352331606217617,
    'Comedy/Spoken':0.6733333333333333,
    'Stage & Screen':0.625,
    'Classical':0.20987654320987653,
    'Easy Listening':0.3333333333333333,
    'Avant-Garde':0.1891891891891892,
    'Unknown':0.0,
    "Children's":0.3333333333333333
}

In [2]:
frame_1=origin_data[['follower_main_genre','influencer_main_genre']]
group_1=frame_1.groupby('follower_main_genre')
dict_1={}
for key,df in group_1:
    dict_1[key]=df
key = ['Pop/Rock', 'R&B;', 'Country', 'Jazz', 'Electronic', 'Vocal', 'Reggae', 'Latin', 'Folk', 'Blues',
         'Religious', 'International', 'New Age', 'Comedy/Spoken', 'Stage & Screen', 'Classical', 'Easy Listening',
         'Avant-Garde', 'Unknown', "Children's"]
temp=dict_1['Pop/Rock']

In [3]:
dict_2={}
for temp_key in key:
    temp=dict_1[temp_key]
    total_sum=temp.shape[0]
    temp_1=temp.groupby('influencer_main_genre').count()
    temp_2=temp_1.apply(lambda x:x/total_sum,axis=1)
    temp_start=temp_key
    temp_end=list(temp_2.index.values)
    corr=temp_2['follower_main_genre'].tolist()
    for i in range(len(corr)):
        dict_2[(temp_start,temp_end[i])]=corr[i]

In [4]:
frame_2=origin_data[['follower_id','follower_main_genre','influencer_id','influencer_main_genre']]

In [5]:
start=origin_data['influencer_id'].tolist()
end=origin_data['follower_id'].tolist()
pure_follower = list(set(end)-set(start))#纯粹的追随者（即不影响他人）
# temp_frame_1=frame_2[frame_2['follower_id'].isin(pure_follower)]
pure_influencer = list(set(start)-set(end))#纯粹的影响者（即不追随别人）
mix_songer=list(set(start+end)-set(pure_follower)-set(pure_influencer))#既是追随者又是影响者
pure_follower.sort()
pure_influencer.sort()
mix_songer.sort()

In [6]:
'''计算纯影响者（三类艺术家）每人共影响人数'''
temp_frame_1=frame_2[frame_2['influencer_id'].isin(pure_influencer)]
temp_result1=pd.DataFrame(temp_frame_1.groupby('influencer_id')['follower_id'].count())
id1=list(temp_result1.index.values)
id_influence1=temp_result1['follower_id'].tolist()
id_influence_dict1={}
for i in range(len(id1)):
    id_influence_dict1[str(id1[i])]=id_influence1[i]

In [7]:
'''影响者同时追随者（二类艺术家） 每人共影响人数，'''
temp_frame_2=frame_2[frame_2['influencer_id'].isin(mix_songer)]
temp_result2=pd.DataFrame(temp_frame_2.groupby('influencer_id')['follower_id'].count())
id=list(temp_result2.index.values)
id_influence=temp_result2['follower_id'].tolist()
id_influence_dict={}
for i in range(len(id)):
    id_influence_dict[str(id[i])]=id_influence[i]

In [8]:
'''计算每年影响多少人'''
temp_frame_3=origin_data[['influencer_active_start','follower_id']]
temp_result3=pd.DataFrame(temp_frame_3.groupby('influencer_active_start')['follower_id'].count())
year=list(temp_result3.index.values)
year_influence=temp_result3['follower_id'].tolist()
influence_dict={}
for i in range(len(year)):
    influence_dict[str(year[i])]=year_influence[i]

In [9]:
'''计算每人能投出多少票'''
#对于纯追随者（一类艺术家）：
score1=1/len(pure_follower)
dict1={}
for i in pure_follower:
    dict1[str(i)]=score1
'''对于 影响者同时追随者（二类艺术家）'''
temp_frame_4=origin_data[['influencer_id','influencer_active_start']]
temp_frame_4=temp_frame_4[temp_frame_4['influencer_id'].isin(mix_songer)]
temp_frame_5=temp_frame_4.drop_duplicates(subset=['influencer_id','influencer_active_start'], keep='first')
mix_dict={}
for i in range(temp_frame_5.shape[0]):
    temp_id=str(temp_frame_5.iloc[i,0])
    temp_year=str(temp_frame_5.iloc[i,1])
    person_influence=id_influence_dict[temp_id]
    jubu_yaer=influence_dict[temp_year]
    zhi=person_influence/jubu_yaer
    mix_dict[temp_id]=zhi

In [10]:
new_dict=id_influence_dict1
new_dict.update(id_influence_dict)
score_dict=dict1
score_dict.update(mix_dict)
new_data=pd.read_csv('TempData/influence.csv',encoding="ANSI")

In [11]:
fenzhi1=new_data[new_data['follower_id'].isin(pure_follower)]
d=fenzhi1.groupby(['follower_id','Same'])['influencer_count'].sum()
d=pd.DataFrame(d)
d.columns=['total_people']
d.to_csv("TempData/temp02.csv",encoding="utf-8")

In [12]:
d=pd.read_csv("TempData/temp02.csv",encoding='utf-8')
new_pure=pd.merge(fenzhi1,d,on=['follower_id','Same'],how='inner')
new_pure.to_csv("TempData/temp03.csv",encoding='utf-8',index=False)

In [13]:
result={}
group=new_pure.groupby('follower_id')
for id,data in group:
    id=str(id)
    type = str(data.iloc[0,6])
    total_score=score_dict[id]
    bilv=Share[type]
    for i in range(data.shape[0]):
        end=str(data.iloc[i,0])
        start=str(data.iloc[i,4])
        same=str(data.iloc[i,9])
        fenyingxiang=data.iloc[i,8]
        zongyingxiang=data.iloc[i,10]
        if (same=='True'):
            score=total_score*bilv*fenyingxiang/zongyingxiang
        result[(start,end)]=score
print(len(result))

9928


In [14]:
'''最后一部分'''
fenzhi2=new_data[new_data['follower_id'].isin(mix_songer)]
e=fenzhi2.groupby(['follower_id','Same'])['influencer_count'].sum()
e=pd.DataFrame(e)
e.columns=['total_people']
e.to_csv("TempData/temp04.csv",encoding="utf-8")

In [15]:
e=pd.read_csv("TempData/temp04.csv",encoding='utf-8')
new_pure2=pd.merge(fenzhi2,e,on=['follower_id','Same'],how='inner')
new_pure2.to_csv("TempData/temp05.csv",encoding='utf-8',index=False)

In [16]:
result2={}
group2=new_pure2.groupby('follower_id')
for id,data in group2:
    id=str(id)
    type = str(data.iloc[0,6])
    total_score=score_dict[id]
    bilv=Share[type]
    for i in range(data.shape[0]):
        end=str(data.iloc[i,0])
        start=str(data.iloc[i,4])
        same=str(data.iloc[i,9])
        fenyingxiang=data.iloc[i,8]
        zongyingxiang=data.iloc[i,10]
        if (same=='True'):
            score=total_score*bilv*fenyingxiang/zongyingxiang
        result2[(start,end)]=score
print(len(result2))

32842


In [17]:
result.update(result2)
print(len(result))

42770


In [18]:
temp=list(result.keys())
start=[]
end=[]
for i in range(len(temp)):
    st,en=temp[i]
    start.append(st)
    end.append(en)
score=list(result.values())
final=pd.DataFrame({
    'start':start,
    'end':end,
    'score':score
})
final.to_csv("TempData/first_question.csv",encoding='utf-8',index=False)

In [19]:
chuli=final.groupby('end')['score'].sum()
chuli=pd.DataFrame(chuli)
chuli.to_csv("TempData/first_sort.csv",encoding="utf-8")